# Iterator from sequences

In [1]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    def __init__(self, text) -> None:
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self) -> str:
        return 'Sentences(%s)' % reprlib.repr(self.text)

s = Sentence('"The time has come, " the Walrus said.')
list(s) # ['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

Sentences('"The time ha... Walrus said.')

# The `iter()` function

Whenever python needs to iterate over an object `x`, it will call the `iter(x)` function. This function does:
- verify if the `__iter__` method is implemented. If so, calls it
- if `__iter__` is not implemented, but `__getitem__` is, then `iter()` will create an iterator that tries to search by the index, starting with zero
- if that fails, a `TypeError` is raised

This is a extreme example of duck typing. For goose typing, a iterable is only a subclass of `abc.Iterable` if implements the `__iter__` function. This is ensured by the `__subclasshook__`.

## Using iter with a callable

We can use the `iter()` function with two arguments: a callable object (e. g. a function) and a sentinel value, used to create a StopIteration exception

```python
def d6():
    return randint(1, 6)
d6_iter = iter(d6, 1) # <callable_iterator at 0x69>
for roll in d6_iter:
    print(rool) # When 1 is returned, the iteration stops
# 4, 3, 6, 3
```

This only works when the callable receives no arguments.

# Iterable vs iterators

A iterable is any object from which the `iter()` function can create a iterator. Any object that implements `__iter__` is iterable. Any sequence is iterable. Any object that implements `__getitem__` and support the index 0 is iterable.

The best way to check if a object is a iterator is calling `ìsinstance(x, abc.Iterator)`